In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
from extract.locm2 import LOCM2
from extract.polocm import POLOCM
from traces import *
from observation import  ObservedTraceList, ActionObservation
from typing import defaultdict

In [29]:
raw_traces = [
    '(build_fences cattle worker2 worker1 worker2 round1),(collect_resource worker1 noworker worker2 round1 act_clay clay),(ag__finish_round_backhome round1 worker2),(ag__finish_round_renew round1 worker4),(ag__advance_round_normal round1 round2 act_sow),(collect_resource worker2 worker1 worker2 round2 act_reed reed),(take_food worker1 noworker worker2 round2 num0 num1),(ag__finish_round_backhome round2 worker2),(ag__finish_round_renew round2 worker5),(ag__advance_round_normal round2 round3 act_family),(build_fences boar worker2 worker1 worker2 round3),(take_grain worker1 noworker worker2 round3 carrot),(ag__finish_round_backhome round3 worker2),(ag__finish_round_renew round3 worker1),(ag__advance_round_normal round3 round4 act_sheep),(take_grain worker2 worker1 worker2 round4 carrot),(collect_resource worker1 noworker worker2 round4 act_reed reed),(ag__finish_round_backhome round4 worker2),(ag__finish_round_renew round4 worker4),(ag__harvest_collect_end round4 stage1)',
    '(collect_resource worker2 worker1 worker2 round1 act_wood wood),(take_grain worker1 noworker worker2 round1 grain),(ag__finish_round_backhome round1 worker2),(ag__finish_round_renew round1 worker4),(ag__advance_round_normal round1 round2 act_family),(take_food worker2 worker1 worker2 round2 num0 num1),(plow_field worker1 noworker worker2 round2),(ag__finish_round_backhome round2 worker2),(ag__finish_round_renew round2 worker2),(ag__advance_round_normal round2 round3 act_fences),(take_grain worker2 worker1 worker2 round3 grain),(collect_resource worker1 noworker worker2 round3 act_stone stone),(ag__finish_round_backhome round3 worker2),(ag__finish_round_renew round3 worker3),(ag__advance_round_normal round3 round4 act_sheep),(collect_resource worker2 worker1 worker2 round4 act_reed reed),(collect_resource worker1 noworker worker2 round4 act_wood wood),(ag__finish_round_backhome round4 worker2),(ag__finish_round_renew round4 worker2),(ag__harvest_collect_end round4 stage1)'
]
traces = []
for raw_trace in raw_traces:
    steps = []
    for i, raw_step in enumerate(raw_trace.split(',')):
        action = raw_step.strip('()').split(' ')
        action_name = action[0]
        obj_names = action[1:]
        objs = [PlanningObject('na', obj) for obj in obj_names]
        action = Action(action_name, objs)
        step = Step({}, action, i)
        steps.append(step)
    trace = Trace(steps)
    traces.append(trace)

tracelist = TraceList(traces)
obs_tracelist = tracelist.tokenize(ActionObservation, ObservedTraceList)

sorts = LOCM2._get_sorts(obs_tracelist)
merge_sorts = defaultdict(list)
for obj, sort in sorts.items():
    merge_sorts[sort].append(obj)
for sort, objs in merge_sorts.items():
    print(sort, objs)

print()
sorts = POLOCM._get_sorts(obs_tracelist)
merge_sorts = defaultdict(list)
for obj, sort in sorts.items():
    merge_sorts[sort].append(obj)
for sort, objs in merge_sorts.items():
    print(sort, objs)


1 ['boar', 'cattle']
2 ['act_wood', 'clay', 'act_clay', 'act_stone', 'worker2', 'worker4', 'round3', 'wood', 'reed', 'worker3', 'round2', 'worker1', 'round4', 'act_reed', 'stone', 'noworker', 'round1', 'worker5']
3 ['act_sow', 'act_fences', 'act_family', 'act_sheep']
4 ['num0']
5 ['num1']
6 ['grain', 'carrot']
7 ['stage1']

1 ['round1', 'round2', 'round3', 'round4']
2 ['noworker', 'worker2', 'worker3', 'worker4', 'worker5', 'worker1']
3 ['num1']
4 ['reed', 'clay', 'wood', 'stone']
5 ['grain', 'carrot']
6 ['num0']
7 ['act_wood', 'act_clay', 'act_reed', 'act_stone']
8 ['stage1']
9 ['act_sow', 'act_fences', 'act_family', 'act_sheep']
10 ['boar', 'cattle']
0 ['zero']


In [30]:
def get_h_sorts(sorts, obs_tracelist):
    ap_sorts = defaultdict(set)
    for trace in obs_tracelist:
        for obs in trace:
            action = obs.action
            for i, obj in enumerate(action.obj_params):
                sort = sorts[obj.name]
                ap_sorts[(action.name, i+1)].add(sort)
    print(ap_sorts)

h_sorts = get_h_sorts(sorts, obs_tracelist)


defaultdict(<class 'set'>, {('build_fences', 1): {10}, ('build_fences', 2): {2}, ('build_fences', 3): {2}, ('build_fences', 4): {2}, ('build_fences', 5): {1}, ('collect_resource', 1): {2}, ('collect_resource', 2): {2}, ('collect_resource', 3): {2}, ('collect_resource', 4): {1}, ('collect_resource', 5): {7}, ('collect_resource', 6): {4}, ('ag__finish_round_backhome', 1): {1}, ('ag__finish_round_backhome', 2): {2}, ('ag__finish_round_renew', 1): {1}, ('ag__finish_round_renew', 2): {2}, ('ag__advance_round_normal', 1): {1}, ('ag__advance_round_normal', 2): {1}, ('ag__advance_round_normal', 3): {9}, ('take_food', 1): {2}, ('take_food', 2): {2}, ('take_food', 3): {2}, ('take_food', 4): {1}, ('take_food', 5): {6}, ('take_food', 6): {3}, ('take_grain', 1): {2}, ('take_grain', 2): {2}, ('take_grain', 3): {2}, ('take_grain', 4): {1}, ('take_grain', 5): {5}, ('ag__harvest_collect_end', 1): {1}, ('ag__harvest_collect_end', 2): {8}, ('plow_field', 1): {2}, ('plow_field', 2): {2}, ('plow_field', 3)

In [31]:
ap_objs = defaultdict(set)
for trace in obs_tracelist:
    for obs in trace:
        action = obs.action
        for i, obj in enumerate(action.obj_params):
            ap_objs[(action.name, i+1)].add(obj.name)
print(ap_objs)

defaultdict(<class 'set'>, {('build_fences', 1): {'boar', 'cattle'}, ('build_fences', 2): {'worker2'}, ('build_fences', 3): {'worker1'}, ('build_fences', 4): {'worker2'}, ('build_fences', 5): {'round3', 'round1'}, ('collect_resource', 1): {'worker1', 'worker2'}, ('collect_resource', 2): {'worker1', 'noworker'}, ('collect_resource', 3): {'worker2'}, ('collect_resource', 4): {'round2', 'round3', 'round4', 'round1'}, ('collect_resource', 5): {'act_wood', 'act_clay', 'act_reed', 'act_stone'}, ('collect_resource', 6): {'reed', 'clay', 'wood', 'stone'}, ('ag__finish_round_backhome', 1): {'round2', 'round3', 'round4', 'round1'}, ('ag__finish_round_backhome', 2): {'worker2'}, ('ag__finish_round_renew', 1): {'round2', 'round3', 'round4', 'round1'}, ('ag__finish_round_renew', 2): {'worker3', 'worker1', 'worker2', 'worker4', 'worker5'}, ('ag__advance_round_normal', 1): {'round2', 'round3', 'round1'}, ('ag__advance_round_normal', 2): {'round2', 'round3', 'round4'}, ('ag__advance_round_normal', 3):